<a href="https://colab.research.google.com/github/JP-SystemsX/24cast/blob/master/AI_Grid_Hackathon_JP_DEV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
!pip install -q wget
!pip install -q autogluon
!pip install -q pyarrow==15.0.2
!pip install -q pandas

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.0/428.0 kB 28.0 MB/

In [2]:
!wget https://zenodo.org/records/3899018/files/LoadProfile_20IPs_2016.csv -q
!wget https://zenodo.org/records/3899018/files/LoadProfile_30IPs_2017.csv -q

In [3]:
!ls

LoadProfile_20IPs_2016.csv  LoadProfile_30IPs_2017.csv	sample_data


In [4]:
import pandas as pd
from copy import deepcopy
import numpy as np

In [5]:
lp_20ips_2016 = pd.read_csv("LoadProfile_20IPs_2016.csv", sep=";", skiprows=1)
lp_30ups_2017 = pd.read_csv("LoadProfile_30IPs_2017.csv", sep=";", skiprows=1)
lp_20ips_2016.head()

,Time stamp,LG 1,LG 2,LG 3,LG 4,LG 5,LG 6,LG 7,LG 8,LG 9,...,LG 11,LG 12,LG 13,LG 14,LG 15,LG 16,LG 17,LG 18,LG 19,LG 20
0,01.01.2016 00:15:00,39,332.16,10.552,416.0,13.12,0.12,6.0,120,70,...,0.0,27.400,29.6,65.0,2462.4,6.200,0.96,25.8,11.46,220
1,01.01.2016 00:30:00,41,321.36,10.300,418.4,13.12,0.24,6.0,116,74,...,0.0,25.448,30.4,66.0,2462.4,6.200,0.90,34.2,11.32,224
2,01.01.2016 00:45:00,39,336.48,10.348,419.6,13.76,0.12,6.0,108,68,...,0.0,27.000,30.4,68.0,2459.2,5.876,0.90,38.4,11.62,226
3,01.01.2016 01:00:00,40,319.20,10.600,422.0,13.44,0.12,8.0,112,80,...,0.0,27.552,29.6,67.0,2451.2,6.200,1.02,19.8,9.40,220
4,01.01.2016 01:15:00,39,322.80,10.148,418.8,13.44,0.24,6.0,120,72,...,272.0,24.900,31.2,66.0,2436.8,6.200,0.90,16.8,10.58,224


In [6]:
lp_30ups_2017.head()

,Time stamp,LG 01,LG 02,LG 03,LG 04,LG 05,LG 06,LG 07,LG 08,LG 09,...,LG 21,LG 22,LG 23,LG 24,LG 25,LG 26,LG 27,LG 28,LG 29,LG 30
0,01.01.2017 00:15:00,4.340,3.150,0.810,3.560,0.406,7.526,7.913,2.740,0.918,...,19.40,2.1,4.6,6.0,0.853,3.58,14.55,14.55,16.75,12.0
1,01.01.2017 00:30:00,4.510,2.738,0.885,3.530,0.406,6.917,8.150,3.005,1.026,...,19.25,1.8,4.3,8.2,0.805,3.84,14.55,14.55,17.75,12.0
2,01.01.2017 00:45:00,3.795,2.413,0.900,2.785,0.325,6.737,6.938,2.795,0.936,...,19.05,1.9,4.7,5.6,0.805,3.46,16.45,16.45,19.50,11.5
3,01.01.2017 01:00:00,4.245,1.875,0.825,2.760,0.325,6.744,7.088,2.810,0.942,...,19.10,1.8,4.2,6.0,0.817,3.68,14.75,14.75,19.50,11.5
4,01.01.2017 01:15:00,4.050,1.700,0.915,3.020,0.325,7.394,4.075,2.545,0.942,...,19.10,1.8,4.3,8.2,0.817,4.08,14.65,14.65,19.25,11.5


In [7]:
# TODO: Split Data

# AutoGluon

In [8]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
# Prepare Data
df = deepcopy(lp_20ips_2016)#[["Time stamp", "LG 1"]]
df.loc[:, ["Time stamp"]] = df["Time stamp"].str.replace(r'[^0-9.: ]', '', regex=True).str.strip()
df.loc[:, ["Time stamp"]] = pd.to_datetime(df["Time stamp"], format='%d.%m.%Y %H:%M:%S')
# TODO Normalize Data else MASE means nothing
# Concat Columns & Assign Ids
df = df.melt(id_vars=['Time stamp'], var_name='item_id', value_name='target')
df

,Time stamp,item_id,target
0,2016-01-01 00:15:00,LG 1,39.0
1,2016-01-01 00:30:00,LG 1,41.0
2,2016-01-01 00:45:00,LG 1,39.0
3,2016-01-01 01:00:00,LG 1,40.0
4,2016-01-01 01:15:00,LG 1,39.0
...,...,...,...
702795,2016-12-31 23:00:00,LG 20,280.0
702796,2016-12-31 23:15:00,LG 20,288.0
702797,2016-12-31 23:30:00,LG 20,284.0
702798,2016-12-31 23:45:00,LG 20,288.0


In [9]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="Time stamp"
)
train_data

target
item_id timestamp                  
LG 1    2016-01-01 00:15:00    39.0
        2016-01-01 00:30:00    41.0
        2016-01-01 00:45:00    39.0
        2016-01-01 01:00:00    40.0
        2016-01-01 01:15:00    39.0
...                             ...
LG 20   2016-12-31 23:00:00   280.0
        2016-12-31 23:15:00   288.0
        2016-12-31 23:30:00   284.0
        2016-12-31 23:45:00   288.0
        2017-01-01 00:00:00   284.0

[702800 rows x 1 columns]

In [10]:
predictor = TimeSeriesPredictor(
    prediction_length=48,
    path="autogluon-m4-hourly",
    target="target",
    eval_metric="MASE",
    freq="15min"
)

# Dev Setting
predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=5*60,
)

Frequency '15min' stored as '15T'
Beginning AutoGluon training... Time limit = 300s
AutoGluon will save models to 'autogluon-m4-hourly'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          0
Memory Avail:       11.27 GB / 12.67 GB (88.9%)
Disk Space Avail:   65.34 GB / 107.72 GB (60.7%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MASE,
 'freq': '15T',
 'hyperparameters': 'light',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 48,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'time_limit': 300,
 'verbosity': 2}

train_data with frequency 'No

In [11]:
# TODO: Smarter Train-Test Split
test = deepcopy(lp_30ups_2017)#[["Time stamp", "LG 1"]]
test.loc[:, ["Time stamp"]] = test["Time stamp"].str.replace(r'[^0-9.: ]', '', regex=True).str.strip()
test.loc[:, ["Time stamp"]] = pd.to_datetime(test["Time stamp"], format='%d.%m.%Y %H:%M:%S')

# TODO: Check whether makes sense (AG can't handle Nans)
test = test.interpolate(method="linear")

# Concat Columns & Assign Ids
test = test.melt(id_vars=['Time stamp'], var_name='item_id', value_name='target')
test_data = TimeSeriesDataFrame.from_data_frame(
    test.reset_index(drop=True),
    id_column="item_id",
    timestamp_column="Time stamp"
)
predictor.evaluate(test_data)

<ipython-input-11-890bfd28e7da>:7: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  test = test.interpolate(method="linear")
data with frequency 'None' has been resampled to frequency '15T'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


{'MASE': -0.28720131749974964}

C:\Users\Jimmy\AppData\Local\Temp\ipykernel_17624\3953822821.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  test = test.interpolate(method="linear")
C:\Users\Jimmy\Documents\UNI\Projects\24cast\venv\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


Time stamp    0
item_id       0
target        0
dtype: int64

In [ ]:
# High Performance Setting
predictor.fit(
    train_data,
    presets="best_quality",
    time_limit=4*60*60,
)

In [12]:
!git add AI_Grid_Hackathon_JP_DEV.ipynb
!git commit -m "Added Git"
!git push origin master

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
